<a href="https://cognitiveclass.ai"><img src = "http://seekvectorlogo.com/wp-content/uploads/2019/01/coursera-vector-logo.png" width = 200> </a>

<a href="https://cognitiveclass.ai"><img src = "http://www.logospng.com/images/80/ibmwatsonlogo-cogx-london-2018-80659.png" width = 200> </a>


# Segmenting and Clustering Neighborhoods in Monterrey and Austin

#### Coursera Capstone Project, Week 3

## Introduction

An analysis is performed for the case of the company worker Luis, who is moving from Monterrey, Mexico to Austin, Texas due to his company requirements. We need to identify what are the current characteristics of the place were Luis lives, and to compare if there is any similar place in Austin, Texas.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

## 1. Download and Explore Dataset

### Load Libraries and Obtain data

Scraping Wikipedia Table is performed with BeautifulSoup and Request libraries

In [1]:
import geocoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import folium

from bs4 import BeautifulSoup # Library to web scrapping wikipedia tables
import requests # Library to get http requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

#pd.options.display.max_columns = 20

Libraries imported.


### Obtaining Latitude and Longitude with Geocoder library

I figured out that the Geocoder library was not getting information with the default geocoder.google() function, insetad of that I found that the 'arcgis' data source was working better and obtained results.

Also the proposed function to obtain lattitude and longitude for the cities was adapted as a defined function to be applied among the resulting dataframe.

In [2]:
### Web Scrapping Wikipedia Table

fileI = 'Coursera Data.xlsx'
folderI = r'C:\Users\H305128\Documents\\'

################################# Monterrey Data ###############################################
## Monterrey Neighborhood dataframe
xl = pd.ExcelFile(folderI+fileI)
dfMty = xl.parse('MTY CP')

### Function to extract Latitude and Longitude for Monterrey Neighborhoods
def GetCoordMty(Neighborhood,CP):
    g = geocoder.arcgis('{} {}'.format(Neighborhood,CP))
    lat_long_coords = g.latlng
    return lat_long_coords

dfMty['LatLong'] = dfMty.apply(lambda x: GetCoordMty(x['Asentamiento▼'],x['Código Postal']), axis=1)
dfMty[['Latitude','Longitude']] = pd.DataFrame(dfMty['LatLong'].values.tolist())
dfMty = dfMty.drop('LatLong', 1)

dfMty.head(10)

,Asentamiento▼,Tipo de Asentamiento,Código Postal,Municipio,Ciudad,Zona,Mapa,Latitude,Longitude
0,Monterrey Centro,Colonia,64000,Monterrey,Monterrey,Urbana,Mapa,25.664332,-100.317274
1,Nuevo Centro de Monterrey,Equipamiento,64018,Monterrey,Monterrey,Urbana,Mapa,25.671916,-100.300973
2,Condominios Constitución,Unidad habitacional,64019,Monterrey,Monterrey,Urbana,Mapa,25.668419,-100.300588
3,Gonzalitos,Colonia,64020,Monterrey,Monterrey,Urbana,Mapa,25.684598,-100.350584
4,Deportivo Obispado,Colonia,64040,Monterrey,Monterrey,Urbana,Mapa,25.678081,-100.340836
5,María Luisa,Colonia,64040,Monterrey,Monterrey,Urbana,Mapa,25.675132,-100.338444
6,Obispado,Colonia,64060,Monterrey,Monterrey,Urbana,Mapa,25.674287,-100.334677
7,Barrio Antiguo,Colonia,64100,Monterrey,Monterrey,Urbana,Mapa,43.502635,-1.454455
8,Nuevo Repueblo,Colonia,64700,Monterrey,Monterrey,Urbana,Mapa,25.656749,-100.302061
9,Roma,Colonia,64700,Monterrey,Monterrey,Urbana,Mapa,25.658876,-100.296769


In [52]:
################################# Austin Data ###############################################
## Austin Neighborhood dataframe
xl = pd.ExcelFile(folderI+fileI)
dfAu = xl.parse('AU Neigh')

### Function to extract Latitude and Longitude for Monterrey Neighborhoods
def GetCoordAu(Neighborhood,City):
    g = geocoder.arcgis('{} {}'.format(Neighborhood,City))
    lat_long_coords = g.latlng
    return lat_long_coords

dfAu['LatLong'] = dfAu.apply(lambda x: GetCoordAu(x['Neighborhood'],x['City']), axis=1)
dfAu[['Latitude','Longitude']] = pd.DataFrame(dfAu['LatLong'].values.tolist())
dfAu = dfAu.drop('LatLong', 1)

dfAu.head()

,Neighborhood,City,State,Latitude,Longitude
0,Bryker Woods,Austin,Texas,30.305007,-97.754204
1,Caswell Heights,Austin,Texas,30.307865,-97.719418
2,Downtown Austin,Austin,Texas,30.265270,-97.746470
3,Eastwoods,Austin,Texas,30.371418,-97.748008
4,Hancock,Austin,Texas,30.297150,-97.726620


### Map visualization with Folium

#### Create a map of Toronto with neighborhoods superimposed on top

Use geopy library to get the latitude and longitude values of Toronto.

In [102]:
### Monterrey Map

address = 'Monterrey, Nuevo Leon'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_mty = location.latitude
longitude_mty = location.longitude
print('The geograpical coordinate of Monterrey are {}, {}.'.format(latitude, longitude))


# create map of Monterrey using latitude and longitude values
map_mty = folium.Map(location=[latitude_mty, longitude_mty], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(dfMty['Latitude'], dfMty['Longitude'], dfMty['Asentamiento▼']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mty)  
    
map_mty

The geograpical coordinate of Monterrey are 30.2711286, -97.7436995.


In [103]:
### Austin Map

address = 'Austin, TX'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_au = location.latitude
longitude_au = location.longitude
print('The geograpical coordinate of Austin, TX are {}, {}.'.format(latitude, longitude))


# create map of Austin using latitude and longitude values
map_au = folium.Map(location=[latitude_au, longitude_au], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(dfAu['Latitude'], dfAu['Longitude'], dfAu['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_au)  
    
map_au

The geograpical coordinate of Austin, TX are 30.2711286, -97.7436995.


#### Define Foursquare Credentials and Version

In [57]:
CLIENT_ID = 'ZURNZQ4NR1HSOQOYFF5BMRCQC2N4K5VV0AGPNU5K1QA4UAHB' # your Foursquare ID
CLIENT_SECRET = 'TLA32TBUQLKQL2HFV0TA4JYRKFKKWMY5RVQUVVIH4E1DQNUI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: ZURNZQ4NR1HSOQOYFF5BMRCQC2N4K5VV0AGPNU5K1QA4UAHB
CLIENT_SECRET:TLA32TBUQLKQL2HFV0TA4JYRKFKKWMY5RVQUVVIH4E1DQNUI


In [58]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## 2. Explore Neighborhoods in Monterrey and Austin

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [59]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 800 # define radius


In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

## Monterrey Data

In [61]:
dfMty.columns.tolist()

['Asentamiento▼',
 'Tipo de Asentamiento',
 'Código Postal',
 'Municipio',
 'Ciudad',
 'Zona',
 'Mapa',
 'Latitude',
 'Longitude']

In [68]:
mty_venues = getNearbyVenues(names=dfMty['Asentamiento▼'],
                                   latitudes=dfMty['Latitude'],
                                   longitudes=dfMty['Longitude']
                                  )

Monterrey Centro
Nuevo Centro de Monterrey
Condominios Constitución
Gonzalitos
Deportivo Obispado
María Luisa
Obispado
Barrio Antiguo
Nuevo Repueblo
Roma
Roma Sur
Tecnológico
Balcones del Carmen
Comercial Ampliación Doctores
Del Carmen
El Maguey
Hacienda San Francisco
Loma Larga
Lomas de San Francisco
Los Doctores
Nuevas Colonias
Pío X
Unión Francisco I Madero
Loma Larga
Los Magueyes
Sertoma
Ancira
Benito Juárez (F-96)
Centro
Independencia
Unión de Colonos A Reyes
Unión Miguel Barrera
Predio Francisco Zarco
Unión Francisco Zarco
Unión J Carlos Camacho
Unión Loma Larga
Unión Luis Echeverría


In [69]:
print(mty_venues.shape)
mty_venues.head(10)

(554, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Monterrey Centro,25.664332,-100.317274,Fiesta Americana,25.665302,-100.317469,Hotel
1,Monterrey Centro,25.664332,-100.317274,Smart Fit,25.665277,-100.316580,Gym / Fitness Center
2,Monterrey Centro,25.664332,-100.317274,Pabellón M,25.665838,-100.317178,Concert Hall
3,Monterrey Centro,25.664332,-100.317274,Auditorio Pabellón M,25.665564,-100.317050,Concert Hall
4,Monterrey Centro,25.664332,-100.317274,La Vaca Argentina,25.665861,-100.316891,Steakhouse
5,Monterrey Centro,25.664332,-100.317274,Liverpool,25.665875,-100.315203,Department Store
6,Monterrey Centro,25.664332,-100.317274,El Muelle de al Lado,25.665796,-100.316961,Seafood Restaurant
7,Monterrey Centro,25.664332,-100.317274,Metapatio,25.666407,-100.318256,Beer Garden
8,Monterrey Centro,25.664332,-100.317274,Club Ejecutivo,25.664745,-100.314160,Lounge
9,Monterrey Centro,25.664332,-100.317274,Panadería El Nopal,25.667687,-100.316219,Bakery


In [70]:
mty_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ancira,3,3,3,3,3,3
Balcones del Carmen,3,3,3,3,3,3
Barrio Antiguo,5,5,5,5,5,5
Centro,2,2,2,2,2,2
Comercial Ampliación Doctores,12,12,12,12,12,12
Condominios Constitución,20,20,20,20,20,20
Del Carmen,14,14,14,14,14,14
Deportivo Obispado,11,11,11,11,11,11
El Maguey,7,7,7,7,7,7


#### Let's find out how many unique categories can be curated from all the returned venues

In [71]:
print('There are {} uniques categories.'.format(len(mty_venues['Venue Category'].unique())))

There are 122 uniques categories.


## Austin Data

In [72]:
dfAu.columns.tolist()

['Neighborhood', 'City', 'State', 'Latitude', 'Longitude']

In [73]:
au_venues = getNearbyVenues(names=dfAu['Neighborhood'],
                                   latitudes=dfAu['Latitude'],
                                   longitudes=dfAu['Longitude']
                                  )

Bryker Woods
Caswell Heights
Downtown Austin
Eastwoods
Hancock
Heritage
Hyde Park
Judges' Hill
Lower Waller Creek
North University
Oakmont Heights
Old Enfield
Old Pecan Street
Old West Austin
Original Austin
Original West University
Pemberton Heights
Ridgelea
Ridgetop
Rosedale
Shoal Crest
West Downtown


In [75]:
print(au_venues.shape)
au_venues.head(10)

(496, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bryker Woods,30.305007,-97.754204,Olive & June,30.307450,-97.751046,Italian Restaurant
1,Bryker Woods,30.305007,-97.754204,Tiny Boxwoods,30.306058,-97.749789,American Restaurant
2,Bryker Woods,30.305007,-97.754204,Kerbey Lane Café,30.308030,-97.750470,Café
3,Bryker Woods,30.305007,-97.754204,Austin Flower Company,30.307787,-97.751224,Flower Shop
4,Bryker Woods,30.305007,-97.754204,Salon Hush,30.307783,-97.752403,Cosmetics Shop
5,Bryker Woods,30.305007,-97.754204,Brykerwood Veterinary Clinic,30.305978,-97.749611,Veterinarian
6,Bryker Woods,30.305007,-97.754204,Define Body & Mind: Austin,30.308680,-97.753964,Recreation Center
7,Bryker Woods,30.305007,-97.754204,Brick Oven on 35th,30.307610,-97.750941,Italian Restaurant
8,Bryker Woods,30.305007,-97.754204,Things Celtic,30.308213,-97.752807,Gift Shop
9,Bryker Woods,30.305007,-97.754204,MedSavers Pharmacy,30.308361,-97.752707,Pharmacy


In [76]:
au_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bryker Woods,23,23,23,23,23,23
Caswell Heights,31,31,31,31,31,31
Downtown Austin,100,100,100,100,100,100
Hancock,3,3,3,3,3,3
Heritage,5,5,5,5,5,5
Hyde Park,15,15,15,15,15,15
Lower Waller Creek,79,79,79,79,79,79
North University,4,4,4,4,4,4
Oakmont Heights,3,3,3,3,3,3


#### Let's find out how many unique categories can be curated from all the returned venues

In [77]:
print('There are {} uniques categories.'.format(len(au_venues['Venue Category'].unique())))

There are 149 uniques categories.


## 3. Analyze Each Neighborhood

### Monterrey Data

In [78]:
# one hot encoding
mty_onehot = pd.get_dummies(mty_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mty_onehot['Neighborhood'] = mty_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mty_onehot.columns[-1]] + list(mty_onehot.columns[:-1])
mty_onehot = mty_onehot[fixed_columns]

mty_onehot.head(10)

,Neighborhood,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Drugstore,Electronics Store,Exhibit,Fast Food Restaurant,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Furniture / Home Store,Garden Center,Gas Station,Gay Bar,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lounge,Market,Mattress Store,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Mountain,Movie Theater,Moving Target,Nightclub,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Pub,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Theater,Toy / Game Store,University,Vegetarian / Vegan Restaurant,Video Game Store,Water Park,Wings Joint,Yoga Studio
0,Monterrey Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Monterrey Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Monterrey Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Monterrey Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Monterrey Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,Monterrey Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Monterrey Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Monterrey Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Monterrey Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [79]:
mty_onehot.shape

(554, 123)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [80]:
mty_grouped = mty_onehot.groupby('Neighborhood').mean().reset_index()
mty_grouped

,Neighborhood,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Drugstore,Electronics Store,Exhibit,Fast Food Restaurant,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Furniture / Home Store,Garden Center,Gas Station,Gay Bar,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lounge,Market,Mattress Store,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Mountain,Movie Theater,Moving Target,Nightclub,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Pub,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Theater,Toy / Game Store,University,Vegetarian / Vegan Restaurant,Video Game Store,Water Park,Wings Joint,Yoga Studio
0,Ancira,0.000000,0.00,0.00,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.333333,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00
1,Balcones del Carmen,0.000000,0.00,0.00,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00
2,Barrio Antiguo,0.000000,0.00,0.00,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

#### Let's confirm the new size

In [82]:
mty_grouped.shape

# mty_grouped.drop('Neighborhood', 1).head(10)

(32, 123)

#### Let's print each neighborhood along with the top 5 most common venues

In [83]:
num_top_venues = 5

for hood in mty_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mty_grouped[mty_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ancira----
                venue  freq
0              Market  0.33
1     Bed & Breakfast  0.33
2  Mexican Restaurant  0.33
3      Mattress Store  0.00
4           Pet Store  0.00


----Balcones del Carmen----
                venue  freq
0            Pharmacy  0.33
1  Mexican Restaurant  0.33
2              Casino  0.33
3           Pet Store  0.00
4         Pastry Shop  0.00


----Barrio Antiguo----
         venue  freq
0  Yoga Studio   0.2
1        Hotel   0.2
2  Supermarket   0.2
3     Pharmacy   0.2
4   Water Park   0.2


----Centro----
                        venue  freq
0  Construction & Landscaping   0.5
1             Automotive Shop   0.5
2              Mattress Store   0.0
3                   Pet Store   0.0
4                 Pastry Shop   0.0


----Comercial Ampliación Doctores----
                venue  freq
0            Pharmacy  0.17
1  Mexican Restaurant  0.17
2   Convenience Store  0.17
3   College Cafeteria  0.08
4              Casino  0.08


----Condominios Constituc

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [84]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [85]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_mty = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_mty['Neighborhood'] = mty_grouped['Neighborhood']

for ind in np.arange(mty_grouped.shape[0]):
    neighborhoods_venues_sorted_mty.iloc[ind, 1:] = return_most_common_venues(mty_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_mty.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ancira,Mexican Restaurant,Bed & Breakfast,Market,Yoga Studio,Drugstore,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop
1,Balcones del Carmen,Pharmacy,Casino,Mexican Restaurant,Yoga Studio,College Quad,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant
2,Barrio Antiguo,Yoga Studio,Supermarket,Water Park,Pharmacy,Hotel,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop
3,Centro,Construction & Landscaping,Automotive Shop,Yoga Studio,Electronics Store,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio
4,Comercial Ampliación Doctores,Mexican Restaurant,Pharmacy,Convenience Store,College Cafeteria,Café,Salad Place,Casino,Pizza Place,Diner,Cosmetics Shop


### Austin Data

In [86]:
# one hot encoding
au_onehot = pd.get_dummies(au_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
au_onehot['Neighborhood'] = au_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [au_onehot.columns[-1]] + list(au_onehot.columns[:-1])
au_onehot = au_onehot[fixed_columns]

au_onehot.head(10)

,Neighborhood,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Cheese Shop,Chinese Restaurant,City,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Dessert Shop,Discount Store,Dive Bar,Dog Run,Donut Shop,Event Service,Fabric Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden Center,Gas Station,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Lake,Latin American Restaurant,Laundromat,Liquor Store,Lounge,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mongolian Restaurant,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoothie Shop,Snack Place,Soup Place,Spa,Sports Bar,Steakhouse,Storage Facility,Sushi Restaurant,Taco Place,Tennis Court,Tex-Mex Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Wine Bar,Wings Joint,Yoga Studio
0,Bryker Woods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bryker Woods,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bryker Woods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bryker Woods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bryker Woods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Bryker Woods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,Bryker Woods,0,0,0,0,0,0,0,0,

In [88]:
au_onehot.shape

(496, 150)

In [89]:
au_grouped = au_onehot.groupby('Neighborhood').mean().reset_index()
au_grouped

,Neighborhood,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Cheese Shop,Chinese Restaurant,City,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Dessert Shop,Discount Store,Dive Bar,Dog Run,Donut Shop,Event Service,Fabric Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden Center,Gas Station,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Lake,Latin American Restaurant,Laundromat,Liquor Store,Lounge,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mongolian Restaurant,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoothie Shop,Snack Place,Soup Place,Spa,Sports Bar,Steakhouse,Storage Facility,Sushi Restaurant,Taco Place,Tennis Court,Tex-Mex Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Wine Bar,Wings Joint,Yoga Studio
0,Bryker Woods,0.043478,0.00,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.043478,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.043478,0.00,0.000000,0.000000,0.00,0.00,0.043478,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.043478,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.043478,0.00,0.000000,0.00,0.000000,0.043478,0.000000,0.00,0.043478,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.086957,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.130435,0.00,0.000000,0.000000,0.043478,0.000000,0.043478,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.043478,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.043478,0.000000,0.00,0.000000,0.043478
1,Caswell Heights,0.000000,0.00,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,0.032258,0.00,0.032258,0.000000,0.0,0.032258,0.00,0.000000,0.00,0.00,0.0,0.000000,0.129032,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.032258,0.00,0.0,0.032258,0.032258,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.032258,0.0,0.032258,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.064516,0.032258,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.032258,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0

In [90]:
au_grouped.shape

(20, 150)

In [91]:
num_top_venues = 5

for hood in au_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = au_grouped[au_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bryker Woods----
                 venue  freq
0                 Park  0.13
1   Italian Restaurant  0.09
2  American Restaurant  0.04
3       Massage Studio  0.04
4       Clothing Store  0.04


----Caswell Heights----
                venue  freq
0            Bus Stop  0.13
1                 Gym  0.06
2      Ice Cream Shop  0.03
3        Optical Shop  0.03
4  Mexican Restaurant  0.03


----Downtown Austin----
                     venue  freq
0              Coffee Shop  0.07
1                    Hotel  0.05
2               Restaurant  0.05
3  New American Restaurant  0.04
4       Italian Restaurant  0.04


----Hancock----
                 venue  freq
0         Intersection  0.33
1          Golf Course  0.33
2                 Park  0.33
3  American Restaurant  0.00
4         Optical Shop  0.00


----Heritage----
                        venue  freq
0      Thrift / Vintage Store   0.2
1  Construction & Landscaping   0.2
2                    Bus Stop   0.2
3                     Dog Run   

In [92]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_au = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_au['Neighborhood'] = au_grouped['Neighborhood']

for ind in np.arange(au_grouped.shape[0]):
    neighborhoods_venues_sorted_au.iloc[ind, 1:] = return_most_common_venues(au_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_au.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bryker Woods,Park,Italian Restaurant,Yoga Studio,Gift Shop,Pizza Place,Pharmacy,Music Store,Massage Studio,Garden Center,Rental Car Location
1,Caswell Heights,Bus Stop,Gym,Ice Cream Shop,Pet Store,Beer Store,Breakfast Spot,Pool Hall,Pool,Pizza Place,Comedy Club
2,Downtown Austin,Coffee Shop,Hotel,Restaurant,New American Restaurant,Italian Restaurant,Bar,Gay Bar,Music Venue,Clothing Store,Lounge
3,Hancock,Golf Course,Intersection,Park,Yoga Studio,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service,Donut Shop,Dog Run
4,Heritage,Construction & Landscaping,Bus Stop,Trail,Thrift / Vintage Store,Dog Run,Dive Bar,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service


## 4. Cluster Neighborhoods

### Monterrey Data

Run *k*-means to cluster the neighborhood into 5 clusters.

In [93]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

mty_grouped_clustering = mty_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mty_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 3, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [94]:
# add clustering labels

#print (neighborhoods_venues_sorted.head())

neighborhoods_venues_sorted_mty.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), left_on='Neighbourhood', right_on='Neighborhood')

#toronto_merged.head() # check the last columns!

In [95]:
print(neighborhoods_venues_sorted_mty.columns.tolist())
neighborhoods_venues_sorted_mty.head()

['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue']


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Ancira,Mexican Restaurant,Bed & Breakfast,Market,Yoga Studio,Drugstore,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop
1,0,Balcones del Carmen,Pharmacy,Casino,Mexican Restaurant,Yoga Studio,College Quad,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant
2,0,Barrio Antiguo,Yoga Studio,Supermarket,Water Park,Pharmacy,Hotel,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop
3,3,Centro,Construction & Landscaping,Automotive Shop,Yoga Studio,Electronics Store,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio
4,0,Comercial Ampliación Doctores,Mexican Restaurant,Pharmacy,Convenience Store,College Cafeteria,Café,Salad Place,Casino,Pizza Place,Diner,Cosmetics Shop


In [97]:
dfMty.columns.tolist()

['Asentamiento▼',
 'Tipo de Asentamiento',
 'Código Postal',
 'Municipio',
 'Ciudad',
 'Zona',
 'Mapa',
 'Latitude',
 'Longitude']

In [99]:
mty_merged = dfMty
# Rename column to match joining dataframes
mty_merged = mty_merged.rename(columns={'Asentamiento▼':'Neighborhood'})

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mty_merged = mty_merged.join(neighborhoods_venues_sorted_mty.set_index('Neighborhood'), on='Neighborhood')

mty_merged.head() # check the last columns!

,Neighborhood,Tipo de Asentamiento,Código Postal,Municipio,Ciudad,Zona,Mapa,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Monterrey Centro,Colonia,64000,Monterrey,Monterrey,Urbana,Mapa,25.664332,-100.317274,0.0,Mexican Restaurant,Hotel,Convenience Store,Restaurant,Concert Hall,Coffee Shop,Seafood Restaurant,Taco Place,Department Store,Pizza Place
1,Nuevo Centro de Monterrey,Equipamiento,64018,Monterrey,Monterrey,Urbana,Mapa,25.671916,-100.300973,0.0,Mexican Restaurant,Convenience Store,Pharmacy,Argentinian Restaurant,Dance Studio,Burger Joint,Comfort Food Restaurant,Italian Restaurant,Burrito Place,Ice Cream Shop
2,Condominios Constitución,Unidad habitacional,64019,Monterrey,Monterrey,Urbana,Mapa,25.668419,-100.300588,0.0,Mexican Restaurant,Convenience Store,Pharmacy,Restaurant,Argentinian Restaurant,Ice Cream Shop,Pizza Place,Plaza,Burrito Place,Italian Restaurant
3,Gonzalitos,Colonia,64020,Monterrey,Monterrey,Urbana,Mapa,25.684598,-100.350584,0.0,Taco Place,Sandwich Place,Wings Joint,Bar,Burger Joint,Mexican Restaurant,Hotel,Cupcake Shop,Pool Hall,Brewery
4,Deportivo Obispado,Colonia,64040,Monterrey,Monterrey,Urbana,Mapa,25.678081,-100.340836,0.0,Convenience Store,Breakfast Spot,Restaurant,Mexican Restaurant,Pool,Hotel,Pizza Place,Plaza,Pharmacy,Food Truck


In [100]:
## Drop nulls

mty_merged = mty_merged.dropna(subset=['Cluster Labels'])

## Convert Cluster Labels to int
mty_merged['Cluster Labels'] = mty_merged['Cluster Labels'].astype(int)

mty_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 0 to 36
Data columns (total 20 columns):
Neighborhood              33 non-null object
Tipo de Asentamiento      33 non-null object
Código Postal             33 non-null int64
Municipio                 33 non-null object
Ciudad                    33 non-null object
Zona                      33 non-null object
Mapa                      33 non-null object
Latitude                  33 non-null float64
Longitude                 33 non-null float64
Cluster Labels            33 non-null int32
1st Most Common Venue     33 non-null object
2nd Most Common Venue     33 non-null object
3rd Most Common Venue     33 non-null object
4th Most Common Venue     33 non-null object
5th Most Common Venue     33 non-null object
6th Most Common Venue     33 non-null object
7th Most Common Venue     33 non-null object
8th Most Common Venue     33 non-null object
9th Most Common Venue     33 non-null object
10th Most Common Venue    33 non-null obje

Finally, let's visualize the resulting clusters

In [105]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters_mty = folium.Map(location=[latitude_mty, longitude_mty], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mty_merged['Latitude'], mty_merged['Longitude'], mty_merged['Neighborhood'], mty_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_mty)
       
map_clusters_mty

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [119]:
mty_merged.loc[mty_merged['Cluster Labels'] == 0, mty_merged.columns[[0] + list(range(5, mty_merged.shape[1]))]]

,Neighborhood,Zona,Mapa,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Monterrey Centro,Urbana,Mapa,25.664332,-100.317274,0,Mexican Restaurant,Hotel,Convenience Store,Restaurant,Concert Hall,Coffee Shop,Seafood Restaurant,Taco Place,Department Store,Pizza Place
1,Nuevo Centro de Monterrey,Urbana,Mapa,25.671916,-100.300973,0,Mexican Restaurant,Convenience Store,Pharmacy,Argentinian Restaurant,Dance Studio,Burger Joint,Comfort Food Restaurant,Italian Restaurant,Burrito Place,Ice Cream Shop
2,Condominios Constitución,Urbana,Mapa,25.668419,-100.300588,0,Mexican Restaurant,Convenience Store,Pharmacy,Restaurant,Argentinian Restaurant,Ice Cream Shop,Pizza Place,Plaza,Burrito Place,Italian Restaurant
3,Gonzalitos,Urbana,Mapa,25.684598,-100.350584,0,Taco Place,Sandwich Place,Wings Joint,Bar,Burger Joint,Mexican Restaurant,Hotel,Cupcake Shop,Pool Hall,Brewery
4,Deportivo Obispado,Urbana,Mapa,25.678081,-100.340836,0,Convenience Store,Breakfast Spot,Restaurant,Mexican Restaurant,Pool,Hotel,Pizza Place,Plaza,Pharmacy,Food Truck
5,María Luisa,Urbana,Mapa,25.675132,-100.338444,0,Gym,Bakery,Breakfast Spot,Dance Studio,Mexican Restaurant,Cupcake Shop,Hotel,Pizza Place,History Museum,Historic Site
6,Obispado,Urbana,Mapa,25.674287,-100.334677,0,Flower Shop,Convenience Store,Mexican Restaurant,Taco Place,Gym,Brewery,Food Truck,Argentinian Restaurant,Food & Drink Shop,Seafood Restaurant
7,Barrio Antiguo,Urbana,Mapa,43.502635,-1.454455,0,Yoga Studio,Supermarket,Water Park,Pharmacy,Hotel,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop
8,Nuevo Repueblo,Urbana,Mapa,25.656749,-100.302061,0,Taco Place,Gym,Mexican Restaurant,Sandwich Place,Burger Joint,Seafood Restaurant,Cocktail Bar,Automotive Shop,Auto Workshop,Construction & Landscaping
9,Roma,Urbana,Mapa,25.658876,-100.296769,0,Taco Place,Juice Bar,Sandwich Place,Burger Joint,Mexican Restaurant,Pizza Place,Breakfast Spot,Convenience Store,Pastry Shop,Theater


#### Cluster 2

In [120]:
mty_merged.loc[mty_merged['Cluster Labels'] == 1, mty_merged.columns[[0] + list(range(5, mty_merged.shape[1]))]]

,Neighborhood,Zona,Mapa,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Unión de Colonos A Reyes,Urbana,Mapa,38.39753,-94.351169,1,Food,Yoga Studio,Drugstore,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio
31,Unión Miguel Barrera,Urbana,Mapa,38.39753,-94.351169,1,Food,Yoga Studio,Drugstore,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio


#### Cluster 3

In [121]:
mty_merged.loc[mty_merged['Cluster Labels'] == 2, mty_merged.columns[[0] + list(range(5, mty_merged.shape[1]))]]

,Neighborhood,Zona,Mapa,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Unión J Carlos Camacho,Urbana,Mapa,38.351883,-94.590489,2,Bar,Diner,Yoga Studio,Electronics Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop


#### Cluster 4

In [122]:
mty_merged.loc[mty_merged['Cluster Labels'] == 3, mty_merged.columns[[0] + list(range(5, mty_merged.shape[1]))]]

,Neighborhood,Zona,Mapa,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Centro,Urbana,Mapa,33.19171,-92.62329,3,Construction & Landscaping,Automotive Shop,Yoga Studio,Electronics Store,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio


#### Cluster 5

In [123]:
mty_merged.loc[mty_merged['Cluster Labels'] == 4, mty_merged.columns[[0] + list(range(5, mty_merged.shape[1]))]]

,Neighborhood,Zona,Mapa,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Unión Luis Echeverría,Urbana,Mapa,-32.622355,-62.679632,4,Auto Garage,Yoga Studio,Electronics Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio


### Austin Data

In [124]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

au_grouped_clustering = au_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(au_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 1, 0, 0, 0, 0, 4, 0])

In [125]:
# add clustering labels

#print (neighborhoods_venues_sorted.head())

neighborhoods_venues_sorted_au.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), left_on='Neighbourhood', right_on='Neighborhood')

#toronto_merged.head() # check the last columns!

In [126]:
print(neighborhoods_venues_sorted_au.columns.tolist())
neighborhoods_venues_sorted_au.head()

['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue']


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Bryker Woods,Park,Italian Restaurant,Yoga Studio,Gift Shop,Pizza Place,Pharmacy,Music Store,Massage Studio,Garden Center,Rental Car Location
1,0,Caswell Heights,Bus Stop,Gym,Ice Cream Shop,Pet Store,Beer Store,Breakfast Spot,Pool Hall,Pool,Pizza Place,Comedy Club
2,0,Downtown Austin,Coffee Shop,Hotel,Restaurant,New American Restaurant,Italian Restaurant,Bar,Gay Bar,Music Venue,Clothing Store,Lounge
3,1,Hancock,Golf Course,Intersection,Park,Yoga Studio,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service,Donut Shop,Dog Run
4,0,Heritage,Construction & Landscaping,Bus Stop,Trail,Thrift / Vintage Store,Dog Run,Dive Bar,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service


In [127]:
dfAu.columns.tolist()

['Neighborhood', 'City', 'State', 'Latitude', 'Longitude']

In [128]:
au_merged = dfAu
# Rename column to match joining dataframes
#au_merged = au_merged.rename(columns={'Asentamiento▼':'Neighborhood'})

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
au_merged = au_merged.join(neighborhoods_venues_sorted_au.set_index('Neighborhood'), on='Neighborhood')

au_merged.head() # check the last columns!

,Neighborhood,City,State,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bryker Woods,Austin,Texas,30.305007,-97.754204,0.0,Park,Italian Restaurant,Yoga Studio,Gift Shop,Pizza Place,Pharmacy,Music Store,Massage Studio,Garden Center,Rental Car Location
1,Caswell Heights,Austin,Texas,30.307865,-97.719418,0.0,Bus Stop,Gym,Ice Cream Shop,Pet Store,Beer Store,Breakfast Spot,Pool Hall,Pool,Pizza Place,Comedy Club
2,Downtown Austin,Austin,Texas,30.265270,-97.746470,0.0,Coffee Shop,Hotel,Restaurant,New American Restaurant,Italian Restaurant,Bar,Gay Bar,Music Venue,Clothing Store,Lounge
3,Eastwoods,Austin,Texas,30.371418,-97.748008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hancock,Austin,Texas,30.297150,-97.726620,1.0,Golf Course,Intersection,Park,Yoga Studio,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service,Donut Shop,Dog Run


In [129]:
## Drop nulls

au_merged = au_merged.dropna(subset=['Cluster Labels'])

## Convert Cluster Labels to int
au_merged['Cluster Labels'] = au_merged['Cluster Labels'].astype(int)

au_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 21
Data columns (total 16 columns):
Neighborhood              20 non-null object
City                      20 non-null object
State                     20 non-null object
Latitude                  20 non-null float64
Longitude                 20 non-null float64
Cluster Labels            20 non-null int32
1st Most Common Venue     20 non-null object
2nd Most Common Venue     20 non-null object
3rd Most Common Venue     20 non-null object
4th Most Common Venue     20 non-null object
5th Most Common Venue     20 non-null object
6th Most Common Venue     20 non-null object
7th Most Common Venue     20 non-null object
8th Most Common Venue     20 non-null object
9th Most Common Venue     20 non-null object
10th Most Common Venue    20 non-null object
dtypes: float64(2), int32(1), object(13)
memory usage: 2.6+ KB


In [131]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters_au = folium.Map(location=[latitude_au, longitude_au], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(au_merged['Latitude'], au_merged['Longitude'], au_merged['Neighborhood'], au_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_au)
       
map_clusters_au

### Examine Clusters

In [132]:
au_merged.loc[au_merged['Cluster Labels'] == 0, au_merged.columns[[0] + list(range(5, au_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bryker Woods,0,Park,Italian Restaurant,Yoga Studio,Gift Shop,Pizza Place,Pharmacy,Music Store,Massage Studio,Garden Center,Rental Car Location
1,Caswell Heights,0,Bus Stop,Gym,Ice Cream Shop,Pet Store,Beer Store,Breakfast Spot,Pool Hall,Pool,Pizza Place,Comedy Club
2,Downtown Austin,0,Coffee Shop,Hotel,Restaurant,New American Restaurant,Italian Restaurant,Bar,Gay Bar,Music Venue,Clothing Store,Lounge
5,Heritage,0,Construction & Landscaping,Bus Stop,Trail,Thrift / Vintage Store,Dog Run,Dive Bar,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service
6,Hyde Park,0,Food Truck,Bakery,Record Shop,Sandwich Place,Park,Cheese Shop,Beer Store,Juice Bar,Italian Restaurant,Pool
8,Lower Waller Creek,0,Food Truck,Bar,Coffee Shop,Mexican Restaurant,Dive Bar,Pizza Place,Ice Cream Shop,Taco Place,Cocktail Bar,Lounge
9,North University,0,Playground,Arcade,Gas Station,Park,Dive Bar,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service,Donut Shop
11,Old Enfield,0,Discount Store,Sandwich Place,City,Chinese Restaurant,Dog Run,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service,Donut Shop
12,Old Pecan Street,0,Pizza Place,Yoga Studio,Snack Place,Paper / Office Supplies Store,Coffee Shop,Dance Studio,Mongolian Restaurant,Pub,Mediterranean Restaurant,Ramen Restaurant
13,Old West Austin,0,Yoga Studio,Spa,Gym / Fitness Center,Gym,Salon / Barbershop,American Restaurant,Grocery Store,Bar,Latin American Restaurant,Liquor Store


In [133]:
au_merged.loc[au_merged['Cluster Labels'] == 1, au_merged.columns[[0] + list(range(5, au_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hancock,1,Golf Course,Intersection,Park,Yoga Studio,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service,Donut Shop,Dog Run


In [134]:
au_merged.loc[au_merged['Cluster Labels'] == 2, au_merged.columns[[0] + list(range(5, au_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Ridgetop,2,Lake,Park,Yoga Studio,Flower Shop,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service,Donut Shop,Dog Run


In [135]:
au_merged.loc[au_merged['Cluster Labels'] == 3, au_merged.columns[[0] + list(range(5, au_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Original Austin,3,Park,Yoga Studio,Dog Run,Flower Shop,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service,Donut Shop,Dive Bar


In [136]:
au_merged.loc[au_merged['Cluster Labels'] == 4, au_merged.columns[[0] + list(range(5, au_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Oakmont Heights,4,Dog Run,Rock Club,Business Service,Donut Shop,Flower Shop,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Service,Yoga Studio


#### References

Data & The World: https://www.dataandtheworld.com/2017/06/20/scraping-wikipedia-tables-python-r/